In [1]:
id = {'KLONGTEI': 'cluster_1892287670_272491964_272492178',
      'RAMA4': 'cluster_272488163_282390730_66263210_66263222',
      'NARANONG': 'cluster_272488164_272492179_3457051443_61907354',
      'SUNLAKAKHON': 'gneJ83',
      'KASEMRAT': 'cluster_272448137_272555800_272555808_7660045934_7710268409',
      'ATTHAKAWI_RAMA4' : '270329335'}
junction_name = list(id.keys())

In [2]:
# import gym
# from gym import error, spaces
import gym
import csv
import os, sys
sys.path.append(os.path.join('/home/ring/sumo-svn/', 'tools'))
import traci
import traci.constants as tc
import numpy as np
from sumolib import checkBinary
import datetime
import matplotlib
import matplotlib.pyplot as plt
import torch
import copy
import random
import pandas as pd
from multiprocessing.pool import ThreadPool
from itertools import zip_longest

In [3]:
namelane_csv = pd.read_csv('namelane_KASEMRAT.csv')
namelane_df = pd.DataFrame(namelane_csv, columns = ['name' , 'id'])
NAME = namelane_df.set_index('name')
ID = namelane_df.set_index('id')
if NAME.loc['KASEMRAT_EB_0_0_XSXX','id'] == '459551209#3_0':
    print('ok')

ok


In [58]:
NAME.loc[:,'id']

name
KASEMRAT_EB_0_0_XSXX           459551209#3_0
KASEMRAT_EB_0_1_XSXX           459551209#3_1
KASEMRAT_EB_0_2_XSXX           459551209#3_2
KASEMRAT_EB_0_3_XSRT           459551209#3_3
KASEMRAT_EB_1_0_LSXX           459551209#0_0
                                  ...       
NARANONG_SW_1_1_XSXX.0       824116549#0_1.0
NARANONG_SW_1_1_XSXX.1       824116549#0_1.1
SUNLAKAKHON_EB_2_2_XSXX.0    824116555#0_2.0
SUNLAKAKHON_EB_2_1_XSXX.0    824116555#0_1.0
SUNLAKAKHON_EB_2_0_XSXX.0    824116555#0_0.0
Name: id, Length: 332, dtype: object

In [5]:
ID.loc['820373198#0_3', 'name']

'RAMA4_WB_2_3_XSXX'

In [6]:
namedetector_csv = pd.read_csv('namedetector_KASEMRAT_flow.csv')
namedetector_df = pd.DataFrame(namedetector_csv, columns = ['name' , 'id'])
NAME_D = namedetector_df.set_index('name')
ID_D = namedetector_df.set_index('id')
listdetector = open("namedetector_KASEMRAT_flow.txt", "r")
detector = {}
for l in listdetector:
    l = l.strip().split(' ')
    if len(l)> 1:
        d = []
        for detec in l[2:]:
            if type(NAME_D.loc[detec,'id']) == str:
                d.append(NAME_D.loc[detec,'id'])
            else : d.append(NAME_D.loc[detec,'id'][0])
    if str(l[0])!= '':
        detector[str(l[0])] = d
list_detector= list(detector.keys())

In [7]:
NAME_D.head()

,id
name,
RAMA4_EB_1_4_XSXX,D459492917#0_4
KLONGTEI_EB_0_0_XSXX,D825786400_0
KLONGTEI_EB_0_1_XSXX,D825786400_1
KLONGTEI_EB_0_2_XSXX,D825786400_2
KLONGTEI_EB_0_3_XSXX,D825786400_3


In [8]:
detector

{'RAMA4_EB_FP1_TP3': ['D459492917#0_4', 'D825786400_4'],
 'RAMA4_EB_FP3_TP2': ['D825786400_0',
  'D825786400_1',
  'D825786400_2',
  'D825786400_3',
  'D825786400_4',
  'D459492912#1_0',
  'D459492912#1_1',
  'D459492912#1_2',
  'D459492912#1_3',
  'D459492912#0_0',
  'D459492912#0_1',
  'D459492912#0_2',
  'D459492912#0_3',
  'D752225467_0',
  'D752225467_1',
  'D825786397_0',
  'D825786397_1',
  'D825786397_2',
  'D825786397_3',
  'D825786397_4',
  'D825786383#0_0',
  'D825786383#0_1',
  'D825786383#0_2',
  'D825786383#0_3',
  'D825786410_0',
  'D825786410_1',
  'D825786410_2',
  'D825786410_3',
  'D459492917#0_0',
  'D459492917#0_1',
  'D459492917#0_2',
  'D459492917#0_3',
  'D459492917#0_4'],
 'RAMA4_WB_FP3_TP2': ['D825786415_5'],
 'RAMA4_WB_FP2_TP4': ['D825786415_5'],
 'RAMA4_SB_FP4_TP5': ['D25053655#0_0', 'D25053655#0_1', 'D25053655#0_2'],
 'RAMA4_NB_FP5_TP1': ['D824116545_0',
  'D824116545_1',
  'D824116545_2',
  'D824116548_1',
  'D824116548_2',
  'D824116548_3',
  'D824116547_

In [9]:
# type(NAME_D.loc['KLONGTEI_EB_0_0_XSXX','id'])
# for i in range(len(list(detector.keys()))):
#     print(i , list(detector.keys())[i])

In [11]:
def get_hot_encoding_current_phase():
    number_phase = [3,7,6,4,3,3]
    current_phase = [traci.trafficlight.getPhase(id[key]) for key in id.keys()]
#     current_phase = [1,1,1,1,1,1]
    if current_phase[0] != 0:
        current_phase[0]+=1
    hot_encoding_current_phase = np.array([])
    for i in range(len(current_phase)):
        binary_phase = np.zeros(number_phase[i])
        if current_phase[i] >0:
            binary_phase[current_phase[i]-1] = 1
        hot_encoding_current_phase = np.concatenate((hot_encoding_current_phase, binary_phase), axis=None)
    return current_phase, hot_encoding_current_phase

In [12]:
# ID.loc['']#, '820373198#0', '820373196#0', '482209831#0']

In [17]:
def get_state_attention(current_phase, hot_encoding_current_phase):
#     current_phase = [KLONGTEI phase, RAMA4 phase,NARANONG phase, SUNLAKAKHON phase, KASEMRAT phase, ATTHAKAWI_RAMA4 phase]
    MAP_RAMA4 = {0: ['RAMA4_EB_FPX_TP1', 'KASEMRAT_EB_FPX_TP2_RAMA4', 'NARANONG_SW_FPX_TP1'],
                 1: ['RAMA4_EB_FP1_TP3', 'RAMA4_NB_FPX_TP5'],
                 2: ['RAMA4_WB_FP2_TP4', 'RAMA4_EB_FPX_TP1', 'RAMA4_NB_FPX_TP5'],
                3: ['RAMA4_EB_FP3_TP2', 'RAMA4_WB_FP3_TP2', 'RAMA4_EB_FPX_TP1', 'RAMA4_NB_FPX_TP5' ],
                4: ['RAMA4_SB_FP4_TP5', 'RAMA4_EB_FPX_TP1', 'RAMA4_NB_FPX_TP5' ],
                5: ['RAMA4_NB_FP5_TP1', 'RAMA4_EB_FPX_TP1'],
                6: ['KASEMRAT_EB_FPX_TP2_RAMA4'],
                7: ['KASEMRAT_EB_FPX_TP2_RAMA4']}
    MAP_KLONGTEI = {0: ['NARANONG_SB_FP5_TP6', 'RAMA4_EB_FP1_TP3'],
                    2: ['RAMA4_EB_FP3_TP2', 'RAMA4_EB_FP1_TP3'],
                    3: ['KLONGTEI_NB_FP3_TPX'],
                   4: ['RAMA4_EB_FP3_TP2',  'RAMA4_EB_FP1_TP3'] }
    MAP_NARANONG = {0: ['RAMA4_NB_FPX_TP5'],
                1: ['NARANONG_SB_FP1_TP2', 'NARANONG_EB_FP1_TP2', 'NARANONG_WB_FPX_TP2', 'NARANONG_WB_FPX_TP2'],
                2: ['NARANONG_WB_FP2_TP3', 'NARANONG_WB_FP2_TP3_FLOW', 'NARANONG_SW_FPX_TP1', 'NARANONG_WB_FPX_TP2'],
                3: ['NARANONG_EB_FP3_TP4', 'NARANONG_SW_FPX_TP1', 'NARANONG_WB_FPX_TP2'],
                4: ['NARANONG_WB_FP4_TP5', 'NARANONG_SW_FPX_TP1', 'NARANONG_WB_FPX_TP2'],
                5: ['NARANONG_SB_FP5_TP6', 'NARANONG_SW_FPX_TP1', 'NARANONG_WB_FPX_TP2'], 
                6: ['NARANONG_NB_FP6_TP1', 'NARANONG_SW_FPX_TP1', 'NARANONG_WB_FPX_TP2'] }
    MAP_SUNLAKAKHON = {0: ['NARANONG_EB_FP3_TP4'],
                    1: ['SUNLAKAKHON_SB_FP1_TPX'],
                    2: ['SUNLAKAKHON_NB_FP2_TP3', 'SUNLAKAKHON_SB_FPX_TP1'],
                    3:['SUNLAKAKHON_EB_FP3_TP4', 'SUNLAKAKHON_WB_FP3_TP4', 'SUNLAKAKHON_SB_FPX_TP1'],
                    4:['SUNLAKAKHON_SB_FP4_TP1', 'SUNLAKAKHON_SB_FPX_TP1'] }
    MAP_KASEMRAT = {0: ['SUNLAKAKHON_SB_FPX_TP1', 'MASUKGRIDLOCK_ARI_NB_FPX_TP2'],
                    1:['KASEMRAT_EB_FPX_TP2', 'KASEMRAT_NB_FPX_TP3'],
                    2:['KASEMRAT_EB_FP2_TP1', 'KASEMRAT_NB_FPX_TP3'],
                    3:['KASEMRAT_EB_FPX_TP2'] }
    MAP_ATTHAKAWI_RAMA4 = {0: ['KASEMRAT_EB_FPX_TP2'],
                           1: ['MASUKGRIDLOCK_SUKHUMVUT_FPX_TP2', 'MASUKGRIDLOCK_ARI_NB_FPX_TP2' ],
                          2:['KASEMRAT_EB_FPX_TP2_RAMA4']}
    MAP = [MAP_KLONGTEI, MAP_RAMA4, MAP_NARANONG, MAP_SUNLAKAKHON,  MAP_KASEMRAT, MAP_ATTHAKAWI_RAMA4]
    state_attention = np.zeros(31)
    for i in range(len(current_phase)):
        for e in MAP[i][current_phase[i]]:
            occupancy = 2#get_occupancy_average_percent(detector[e]) 
            Index_detector = list_detector.index(e) 
            state_attention[Index_detector] = occupancy
    print(state_attention)
    state = np.concatenate((state_attention, hot_encoding_current_phase), axis=None)
    return state


In [18]:
#reset the environment
def start():
    sumoBinary = checkBinary('sumo-gui')
    traci.start([sumoBinary, "-c", "KASEMRAT-SUMO-UsingBookNetFile/osm.sumocfg",
                             "--tripinfo-output", "tripinfo.xml", '--start','true','--quit-on-end','true','--time-to-teleport','-1',
                '--lanechange.duration', '0.1'])

In [19]:
def get_occupancy_average_percent(detector_id): 
    #get occupancy average for all detector in list of detector_id and scale by (Vehicle Length + MinimumGap)/MinimumGap 
    #Vehicle Length = 4.62 MinimumGap = 2.37
    occupancy = (sum([traci.lanearea.getLastStepOccupancy(e) for e in detector_id])/len(detector_id))*((4.62+2.37)/4.62)
    return occupancy

In [51]:
def get_throughput():
    loopID = traci.inductionloop.getIDList()
    throughput = sum([traci.inductionloop.getLastStepVehicleNumber(i) for i in loopID if traci.inductionloop.getLastStepMeanSpeed(id) > 0])
    return throughput

In [21]:
def get_drawback():
    laneID = traci.lane.getIDList()
    drawback = sum([traci.lane.getLastStepVehicleNumber(i) for i in laneID])# NAME.loc[:,'id'])
    return drawback

In [22]:
def get_reward():
    throughput = get_throughput()
    drawback = get_drawback()
    reward = throughput - drawback
    return reward

In [23]:
number_phase = [3,7,6,4,3,3]
def set_phase(action):
    if action < 3:
        phase = action
        traci.trafficlight.setPhase(id[junction_name[0]], phase)
    elif action < 10:
        phase = (action-3)
        traci.trafficlight.setPhase(id[junction_name[1]], phase)
    elif action < 16:
        phase = (action-10)
        traci.trafficlight.setPhase(id[junction_name[2]], phase)
    elif action < 20:
        phase = (action-16)
        traci.trafficlight.setPhase(id[junction_name[3]], phase)
    elif action < 23:
        phase = (action-20)
        traci.trafficlight.setPhase(id[junction_name[4]], phase)
    else:
        phase = (action-23)
        traci.trafficlight.setPhase(id[junction_name[5]], phase)
    return 

In [24]:
reward_memory = []
def plot_durations():
    print('show')
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(reward_memory, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Step')
    plt.ylabel('Reward')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated

In [25]:
start()

In [35]:
action = 1
set_phase(action)

In [36]:
current_phase, hot_encoding_current_phase = get_hot_encoding_current_phase()

In [37]:
get_reward()

0

In [38]:
state = get_state_attention(current_phase, hot_encoding_current_phase)

[2. 2. 0. 0. 0. 0. 2. 0. 0. 2. 2. 0. 0. 2. 0. 0. 0. 2. 2. 0. 0. 0. 0. 0.
 0. 2. 2. 2. 0. 2. 2.]


In [52]:
class MyEnv3(gym.Env):
    def __init__(self, env_config):
        self.step = 0
        self.reward = 0
        self.done = False
        self.reward_memory = []
        self.action_space = gym.spaces.Discrete(25)
        self.observation_space = gym.spaces.Box(low=0, high=100, shape=(62,), dtype=np.float16)
        
    def reset(self):
        
        self.reward_memory.append(self.reward)
        self.step = 0
        start()
        self.reward = 0
        current_phase, hot_encoding_current_phase = get_hot_encoding_current_phase()
        state = get_state_attention(current_phase, hot_encoding_current_phase)
#         print(state)
        self.done = False
        if len(self.reward_memory)%100 == 0:
            print('memory',self.reward_memory[-10:])
        return state 
    
    
    def step(self, action):
        print(action)
        set_phase(action)
        print('ok')
        current_phase, hot_encoding_current_phase = get_hot_encoding_current_phase()
        print('ok2')
        self.reward = get_reward()
        print('ok3')
        state = get_state_attention(current_phase, hot_encoding_current_phase)
        print('ok4')
        if np.isnan(self.reward) == True:
            print('HELP', type(self.reward))
        self.step += 1
#         print('count', self.count)
        self.done = False
        if self.step >= 2880: #2880
            traci.close()
            self.done = True
#         print(np.array(state) , self.reward, self.done, {})
        
        return_state = np.array(state).astype(np.float16)
        return return_state , self.reward, self.done, {}

In [56]:
if __name__ == "__main__":
    print("asd")
    env = MyEnv3(env_config=None)

    for i_episode in range(1):
        observation = env.reset()
        for t in range(1000):
            # env.render()
            print(observation)
            action = env.action_space.sample()
            print('action' ,action)
            observation, reward, done, info = env.step(action)
            if done:
                print("Episode finished after {} timesteps")
                break
    env.close()


asd
[0. 0. 0. 0. 0. 0. 2. 0. 2. 2. 2. 0. 0. 2. 0. 0. 0. 2. 2. 0. 0. 0. 0. 0.
 0. 2. 2. 2. 0. 2. 2.]
[0. 0. 0. 0. 0. 0. 2. 0. 2. 2. 2. 0. 0. 2. 0. 0. 0. 2. 2. 0. 0. 0. 0. 0.
 0. 2. 2. 2. 0. 2. 2. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 1. 0. 0.]
action 2


TypeError: 'int' object is not callable

In [55]:
traci.close()

In [41]:
# import ray
# import ray.tune  as tune
# from ray.rllib.utils import merge_dicts
# from ray.rllib.agents.dqn.dqn import calculate_rr_weights, \
#     DEFAULT_CONFIG as DQN_CONFIG, DQNTrainer, validate_config
# # env = gym.make('CartPole-v0')

# def trial_name_id(trial):
#     return f"{trial.trainable_name}_{trial.trial_id}"

# if __name__ == "__main__":
#     ray.shutdown()
#     ray.init(
# #              object_store_memory=int(1e9),  # 4gb
# #              redis_max_memory=int(1e9)  #2gb
#              )
#     experiment_spec = tune.Experiment(
#         trial_name_creator=trial_name_id,
#         name = "experiment_apex2",
#         run = "APEX",
#         checkpoint_freq = 3,
#         checkpoint_at_end = True,
#         log_to_file=True,
#         config = {
# #         "optimizer": merge_dicts(
# #             DQN_CONFIG["optimizer"], {
# #                 "max_weight_sync_delay": 400,
# #                 "num_replay_buffer_shards": 4,
# #                 "debug": False
# #             }),
# #         "n_step": 3,
#         "num_gpus": 0,
#         "dueling": False,
#         "double_q": False,
#         "num_workers": 3,
#         "buffer_size": 100000,
#         "framework": "tf",
#         "learning_starts": 2160,
#         "train_batch_size": 540,
# #             "num_samples": 20,
#         "rollout_fragment_length": 5,
#         "target_network_update_freq": 540,
#         "prioritized_replay": True,
#         "timesteps_per_iteration": 1080,
# #         "exploration_config": {"type": "PerWorkerEpsilonGreedy"},
# #         "worker_side_prioritization": True,
# #         "min_iter_time_s": 30,
#         # If set, this will fix the ratio of replayed from a buffer and learned
#         # on timesteps to sampled from an environment and stored in the replay
#         # buffer timesteps. Otherwise, replay will proceed as fast as possible.
# #         "training_intensity": None,
#          "worker_side_prioritization": False,
 
# #             "gamma": 0.7,

            
#             "env": MyEnv3 ,

            
#             }
        
#         )
#     tune.run_experiments(experiment_spec)
